# 테스트 api 호출용

In [43]:
import requests
from pprint import pprint
from dotenv import load_dotenv
import os

load_dotenv()

url = 'https://apis.data.go.kr/1471000/HtfsInfoService03/getHtfsItem01?'

# 3) 요청 변수(params) 세팅
params = {
    "pageNo":    "1",          # 페이지 번호
    "numOfRows": "1",         # 한 페이지 결과 수
    "ServiceKey": os.environ.get('DECODING_KEY'), # Swagger에 표시된 정확한 이름(ServiceKey)
    "type":      "json",       # 응답 포맷(xml/json) – default: xml
    # (필요시 API별 추가 파라미터를 여기에 더합니다)
}

# 4) 실제 요청 보내기
response = requests.get(url, params=params, timeout=10)
response.raise_for_status()

# 5) 결과 확인
pprint(response.json())

{'body': {'items': [{'item': {'BASE_STANDARD': '1. 성상 : 고유의 향미가 있고 이미·이취가 없는 '
                                               '노랑 하양색의 입자성이 있는 분말\n'
                                               '2. 프로바이오틱스 수(표시량 '
                                               '300,000,000,000(3,000억) CFU/g) '
                                               ': 표시량 이상\n'
                                               '3. 대장균군 : 음성\n'
                                               '4. 납 : 1.0 mg/kg 이하\n'
                                               '5. 카드뮴 : 0.3 mg/kg 이하',
                              'DISTB_PD': '제조일로부터 24개월까지',
                              'ENTRPS': '일동바이오사이언스(주)',
                              'INTAKE_HINT1': '1. 질환이 있거나 의약품 복용 시 전문가와 '
                                              '상담하십시오.\n'
                                              '2. 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 '
                                              '수 있습니다.\n'
                                              '3. 어

# vector database embedding

In [41]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()
print(pc.list_indexes().names())

PINECONE_INDEX_NAME = 'supplement-rag'
PINECONE_INDEX_REGION = 'us-east-1'
PINECONE_INDEX_CLOUD = 'aws'
PINECONE_INDEX_METRIC = 'cosine'
PINECONE_INDEX_DIMENSION = 1536

OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDING_MODEL = 'text-embedding-3-small'

if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DIMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(
            region=PINECONE_INDEX_REGION,
            cloud=PINECONE_INDEX_CLOUD,
        )
    )
    print(f'{PINECONE_INDEX_NAME} index 생성완료!')
else:
    print(f'{PINECONE_INDEX_NAME} index가 이미 존재합니다.')

['adv-rag-compressed', 'adv-rag-meta', 'winemag-review-data', 'adv-rag']
supplement-rag index 생성완료!


In [42]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from tqdm import tqdm

# 임베딩 모델
embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL)

# 벡터 스토어
vector_store = PineconeVectorStore(
    index_name=PINECONE_INDEX_NAME,
    embedding=embeddings,
)

url = 'https://apis.data.go.kr/1471000/HtfsInfoService03/getHtfsItem01'
params = {
    "pageNo": "1",
    "numOfRows": "100", # 한 페이지에 100개가 최대 -> json 파일로 전체 임베딩 해놓는게 어떨지....
    "ServiceKey": os.getenv("DECODING_KEY"),
    "type": "json"
}
response = requests.get(url, params=params, timeout=10)
response.raise_for_status()
data = response.json()
items = data['body']['items']

# prduct : 상품명
# main_fnctn : 주요기능
# intake_hint : 주의사항
# distb_pd : 유통기한
# base_standard : 성상
# entrps : 제조사
# prsrv_pd : 보관방법

texts = []
metadata_list = []

def safe_get(data, key):
    value = data.get(key)
    return value if value is not None else ""

for item in tqdm(items):
    data = item['item']

    metadata = {
        "상품명": safe_get(data, 'PRDUCT'),
        "주요기능": safe_get(data, 'MAIN_FNCTN'),
        "주의사항": safe_get(data, 'INTAKE_HINT1'),
        "유통기한": safe_get(data, 'DISTB_PD'),
        "성상": safe_get(data, 'BASE_STANDARD'),
        "제조사": safe_get(data, 'ENTRPS'),
        "보관방법": safe_get(data, 'PRSRV_PD')
    }

    page_content = "\n".join(f"{k}: {v}" for k, v in metadata.items())

    doc = Document(page_content=page_content, metadata=metadata)
    texts.append(doc)

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(texts)

vector_store.add_documents(split_docs)

100%|██████████| 100/100 [00:00<00:00, 28826.83it/s]


['eb01b863-3004-4c43-b13e-f91816e1f9b2',
 '519fe154-5baf-4c44-a2a7-2ca88bd946f5',
 '1ef14a73-5d80-4846-a7a7-f7c1ea13e30b',
 'f4e4e109-3ef1-4dc9-94af-e5b320d42b29',
 '9b360233-f480-47e7-a563-97270af4f893',
 'ef6b4a0f-6fd9-4a6f-88ec-269f0f7a21f4',
 '5860851f-6cdd-455c-a148-ff944e078ea2',
 'cc4a621d-feb9-43b5-82a7-87ede5f38efb',
 'c060f61f-0a18-4d41-9807-61026e314232',
 '09f2fb5e-432b-46f1-bbab-8d0a5ccbdb6f',
 '3166a69d-4850-4599-a695-19949e4e93bb',
 '07f2e6ee-3181-4f48-9f44-7ef06d68419f',
 'f134565b-f1a5-448e-bfdc-7cee56361a49',
 '3a18f6ec-9dec-41c4-935a-fe066ee8638a',
 '954e83d1-81e7-43d1-b9b4-68692c17a2d9',
 '79039008-5540-4ac0-b3bb-a1bb2e798343',
 '8bb9201c-44b5-45be-a19a-c0c99d9da22a',
 'a03b82f5-ce7d-4421-a7c6-a56f5c97a5e6',
 '3057af84-0253-4b8f-9408-d800e2e53c14',
 '26e08839-b8f0-4c63-8dea-fa3b956d907e',
 'e90d1c94-eda2-4975-aa66-6e79126ee0d9',
 'b71b4583-8181-4684-9289-8330223c95f8',
 'e792da71-1c63-40e9-b3d0-e41a6764cc07',
 '7311e4ad-507e-4986-8780-07487de628dd',
 '5c86f5f9-70b6-